<a href="https://colab.research.google.com/github/ignacio-chacon/crypto_trading_ai/blob/main/baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install ta

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for ta: filename=ta-0.10.2-py3-none-any.whl size=29103 sha256=6840ea5d2dfe2b0928cf2519b9b82307baf6e55842d729fc626cb1b032927163
  Stored in directory: /root/.cache/pip/wheels/6a/76/03/7f785aaa50b9c6ec7e3fd105a62c1b2c45a034512d51e024a9
Successfully built ta


In [3]:
import pandas as pd
import ta

In [4]:
df_hist_data = pd.concat([pd.read_csv("https://github.com/ignacio-chacon/crypto_trading_ai/raw/main/hist_data_1.csv"),
                         pd.read_csv("https://github.com/ignacio-chacon/crypto_trading_ai/raw/main/hist_data_2.csv")])

In [5]:
df_hist_data.head()

,date,open,high,low,close,Volume BTC,Volume USD
0,2017-01-01 00:00:00,966.34,966.37,966.16,966.37,15.697376,15169.245264
1,2017-01-01 00:05:00,966.43,966.58,966.43,966.58,0.438732,424.053962
2,2017-01-01 00:10:00,966.57,966.57,964.60,965.55,6.661770,6434.262218
3,2017-01-01 00:15:00,965.59,966.57,965.55,965.55,20.773385,20078.145181
4,2017-01-01 00:20:00,965.55,966.54,964.87,964.87,0.670923,647.704320


In [7]:
df_hist_data_ta = ta.add_all_ta_features(df_hist_data,open="open", high="high", low="low", close="close", volume="Volume BTC")

/usr/local/lib/python3.9/dist-packages/ta/trend.py:780: RuntimeWarning: invalid value encountered in double_scalars
  dip[idx] = 100 * (self._dip[idx] / value)
/usr/local/lib/python3.9/dist-packages/ta/trend.py:785: RuntimeWarning: invalid value encountered in double_scalars
  din[idx] = 100 * (self._din[idx] / value)


In [9]:
df_hist_data_ta.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 542925 entries, 0 to 292924
Data columns (total 93 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   date                       542925 non-null  object 
 1   open                       535063 non-null  float64
 2   high                       535063 non-null  float64
 3   low                        535063 non-null  float64
 4   close                      535063 non-null  float64
 5   Volume BTC                 542925 non-null  float64
 6   Volume USD                 542925 non-null  float64
 7   volume_adi                 542925 non-null  float64
 8   volume_obv                 542925 non-null  float64
 9   volume_cmf                 542906 non-null  float64
 10  volume_fi                  542912 non-null  float64
 11  volume_em                  534511 non-null  float64
 12  volume_sma_em              531268 non-null  float64
 13  volume_vpt                 53